In [1]:
import numpy as np
import pandas as pd
import tqdm
import re

In [2]:
ru_topics = {
    'music': ['музыка', 'рок-группа', 'альбом', 'песня', 'звук', 'рок', 'гитара', 'джаз', 'рэп', 'музыкант'],
    'education': ['студент', 'обучение', 'курсы', 'изучение', 'навыки', 'образование', 'учитель', 'школа', 'университет'],
    'politics': ['выборы', 'голосование', 'ассамблея', 'кандидат', 'демократия', 
                 'консул', 'министр', 'парламент', 'политик', 'законодательный', 'республиканский', 'собрание'],
    'sport': ['футбол', 'тренер', 'баскетбол', 'турнир', 'расписание', 'игры', 'лига', 'дивизион', 'команда', 'полузащитник'],
    'business': ['технология', 'платформа', 'компания', 'производство', 'продукт', 'дизайн',
                 'обновление', 'автоматизация', 'пользователь', 'эксплуатация'],
    'literature': ['книга', 'литература', 'художественная', 'толстой', 'шекспир', 'достоевский', 'роман', 'фанфин',
                   'приключение'],
    'crime': ['полиция', 'суд', 'преступник', 'инцидент', 'штраф', 'преступление', 'тюрьма', 'расследование', 
              'расследовать', 'жертва'],
    'travel': ['турист', 'туризм', 'путешествие', 'море', 'каникулы', 'пляж', 'отель', 'хостел', 'отпуск'],
    'games': ['игры', 'xbox', 'игрок', 'steam', 'карты', 'игроки', 'урон', 'switch', 'дракон', 'персонаж', 
              'геймплей', 'консоль'],
    'arhitecture': ['здание', 'построить', 'церковь', 'строение', 'мост', 
                    'крыша', 'улица', 'стиль', 'башня', 'спроектирован', 'камень',
                    'архитектура']
}

In [3]:
big_df_ru = pd.read_csv('/home/mlepekhin/data/big/ru.csv')
big_df_ru.head()

,Unnamed: 0,target,text
0,0,A8,"СИАБ раскидывает розничную сеть , Деловой Пете..."
1,1,A1,Мужчины о женщинах и женщины о мужчинах . Цен...
2,2,A8,Сервисное обслуживание и ремонт « Ивановская ...
3,3,A17,"Представляю вам коллекцию книг для блоггеров ,..."
4,4,A8,"Статьи и заметки Как мы уже сообщали , 26 ноя..."


In [4]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [5]:
def get_sentences_for_topic_ru(topic_words, df, sent_for_topic):
    result = []
    
    for text in df.text.values[:5000]:
        for sentence in re.split(r"[.?!]+", text):
            prefix = ' '.join(sentence.split()[:20]).lower()
            doc = Doc(prefix)
            doc.segment(segmenter)
            doc.tag_morph(morph_tagger)
            for token in doc.tokens:
                token.lemmatize(morph_vocab)
                if token.lemma in topic_words:
                    result.append(prefix)
                    break
            if len(result) == sent_for_topic:
                return result
    return result


def make_topic_sentences_df(topic_dict, df, sent_for_topic=100, get_sentences_for_topic=None):
    result_df = pd.DataFrame({'topic': [], 'sentence': []})
    for topic, topic_words in tqdm.tqdm(topic_dict.items()):
        for sentence in get_sentences_for_topic(set(topic_words), df, sent_for_topic):
            result_df = result_df.append({'topic': topic, 'sentence': sentence}, ignore_index=True)
    return result_df

In [6]:
sent_df_ru = make_topic_sentences_df(ru_topics, big_df_ru, get_sentences_for_topic=get_sentences_for_topic_ru)

100%|██████████| 10/10 [07:27<00:00, 44.78s/it]


In [7]:
print(sent_df_ru.shape)
sent_df_ru.head()

(1000, 2)


,topic,sentence
0,music,на тумбочке слева от шкафа рассматриваем альбо...
1,music,любое другое использование песен без дополните...
2,music,"как правило , изобразительному искусству и муз..."
3,music,детям частенько говорят : не занимайся музыкой...
4,music,"обновлен раздел "" фотографии "" - добавлено нес..."


In [8]:
sent_df_ru.to_csv('/home/mlepekhin/data/ru_topic_big_sentences.csv')

## English

In [9]:
big_df_en = pd.read_csv('/home/mlepekhin/data/big/en.csv')
big_df_en.head()

,Unnamed: 0,target,text
0,0,A17,The ' Baronet Supreme ' is upholstered using ...
1,1,A17,This book explores some of the remarkable sto...
2,2,A17,Course Aims This course will teach support pr...
3,3,A7,In the News 24 November 2002 The Office of Fa...
4,4,A7,Signing the metadata for the SDSS federation ...


In [10]:
en_topics = {
    'music': ['music', 'band', 'album', 'songs', 'sound', 'love', 'rock', 'playing', 'guitar', 'jazz'],
    'education': ['student', 'learning', 'courses', 'teaching', 'skills', 'education', 'study', 'college'],
    'politics': ['election', 'votes', 'assembly', 'candidate', 'democratic', 
                 'council', 'minister', 'parliament', 'politician', 'legislative', 'seats', 'vote'],
    'sport': ['football', 'coach', 'basketball', 'tournament', 'schedule', 'games', 'league', 'division', 'team', 'teams'],
    'business': ['technology', 'platform', 'companies', 'industry', 'product', 'design',
                 'upgrade', 'automation', 'users', 'ideas'],
    'literature': ['books', 'literature', 'fiction', 'tolstoy', 'shakespeare', 'dostoevsky', 'romanism', 'fanfics', 'adventure'],
    'crime': ['police', 'court', 'officer', 'incident', 'charges', 'crime', 'prison', 'investigation', 'accused', 'victim'],
    'travel': ['tourist', 'tourism', 'travel', 'seaside', 'vacation', 'beach', 'hotel', 'hostel'],
    'games': ['games', 'xbox', 'players', 'steam', 'cards', 'player', 'damage', 'switch', 'dragon', 'character', 
              'reload', 'console'],
    'arhitecture': ['building', 'historic', 'church', 'buildings', 'brick', 
                    'roof', 'street', 'style', 'tower', 'designed', 'stone', 'architecture']
}

In [11]:
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 

print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora")) 

rocks : rock
corpora : corpus


In [12]:
def get_sentences_for_topic_en(topic_words, df, sent_for_topic):
    result = []
    
    for text in df.text.values[:10000]:
        for sentence in re.split(r"[.?!]+", text):
            prefix_tokens = sentence.lower().split()[:20]
            for token in prefix_tokens:
                if lemmatizer.lemmatize(token) in topic_words:
                    result.append(' '.join(prefix_tokens))
                    break
            if len(result) == sent_for_topic:
                return result
    return result

In [13]:
sent_df_en = make_topic_sentences_df(en_topics, big_df_en, get_sentences_for_topic=get_sentences_for_topic_en)

100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


In [14]:
print(sent_df_en.shape)
sent_df_en.head()

(1000, 2)


,topic,sentence
0,music,at upton heath school we aim to give the child...
1,music,music on a summer evening with ronan keating i...
2,music,do n't you just love it when a plan comes toge...
3,music,dont got buying stuff from billy bobs corner m...
4,music,"an all-star cast including hugh grant , emma t..."


In [15]:
sent_df_en.to_csv('/home/mlepekhin/data/en_topic_big_sentences.csv')